In [1]:
import sys
sys.path.insert(0, "../app/coman")
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
from ActuaDocumentLoader import ActuaDocumentLoader
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
)
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

In [21]:
load_dotenv()

index_name="test-index-tom"

model = AzureChatOpenAI(
    openai_api_version=str(os.getenv("AZURE_OPENAI_API_VERSION")),
    azure_deployment=str(os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")),
)

AZURE_SEARCH_KEY = str(os.getenv("AZURE_SEARCH_KEY"))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="orisai-text-embedding-3-large-development",
)

retriever = AzureAISearchRetriever(
    content_key="content", index_name=index_name, service_name="orisai-search-development", api_key=AZURE_SEARCH_KEY, top_k=1
)

In [9]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
model.invoke([message])


AIMessage(content="J'aime la programmation.", response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 19, 'total_tokens': 26}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-b79176fe-5c10-4274-84d3-0c5d33ce8f72-0')

In [18]:

retriever.get_relevant_documents("Wat is de woonfiscaliteit in brussel?")

[Document(page_content='Nooit eerder verhuisden zoveel Brusselaars naar Vlaanderen <ul><li><strong>Recordaantal van 24.000 Brusselaars verhuisden vorig jaar naar Vlaanderen</strong></li><li><strong>Hoge prijzen in Brussel drijven inwoners naar Vlaamse rand en Vlaanderen.</strong></li><li><strong>Effect van corona en gewijzigde woonwensen stevig zichtbaar in verhuisbewegingen 2021.</strong></li><li><strong>Ook fiscale factoren doen Brusselaars voor Vlaanderen kiezen.</strong></li><li><strong>Brusselse huurmarkt eveneens onder druk door krapte.&nbsp;</strong></li></ul><p><strong>Nog nooit verhuisden zoveel Brusselaars naar Vlaanderen als in 2021. Vorig jaar verhuisden maar liefst 24.000 Brusselaars naar Vlaanderen, een recordaantal.&nbsp;Dat stelt CIB Vlaanderen, die de jaarlijkse cijfers van de&nbsp;FOD Binnenlandse zaken onderzocht. De stadsvlucht na corona en toegenomen vraag naar woningen met tuin of terras zijn volgens de sectorvertegenwoordiger van de vastgoedsector de belangrijkst

In [27]:
# Prompt template
template = """Answer the question based on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in chain.astream("Wat is de woonfiscaliteit in brussel?"):
    print(chunk, flush=True)

|De| w|oon|f|iscal|iteit| in| Br|uss|el| wordt| be|pa|ald| door| het| Br|uss|else| Gew|est|.| De| fis|cale| fact|oren| kunnen| een| inv|lo|ed| hebben| op| de| bes|l|issing| van| Br|uss|ela|ars| om| te| ver|hu|izen| naar| andere| gew|est|en| zoals| V|la|ander|en|.| In| het| document| wordt| gen|o|emd| dat| het| Br|uss|else| Gew|est| de| hoog|ste| registr|ati|ere|chten| h|ante|ert|.| In| V|la|ander|en| beta|alt| men| |3|%| registr|ati|ere|chten|,| ter|wij|l| men| in| Br|uss|el| |12|,|5|%| beta|alt|.| 

|Br|uss|el| comp|ense|ert| dit| wel| door| de| z|og|ena|am|de| ab|at|tement|en|,| wat| bet|ek|ent| dat| er| een| som| is| waar|op| geen| bel|asting| wordt| beta|ald|.| Op| dit| moment| hoe|ft| er| in| Br|uss|el| geen| bel|asting| te| worden| beta|ald| op| de| eerste| |175|.|000|

CancelledError: 